# ------------------------------ Projeto Cetus ----------------------------------

### Assunto: Benefícios ao Cidadão -  Bolsa Família
### Defasagem: M-4

In [18]:
# Autor: Bruno Jardim
# Data: 10/03/2019
# Descrição: Código para salvar os dados do portal transparência no datalake (AWS - S3)
# URL(dados): http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos
# URL(Metadados): http://www.portaltransparencia.gov.br/pagina-interna/603397-dicionario-de-dados-bolsa-familia-pagamentos
# URL(Periodicidade): http://www.portaltransparencia.gov.br/beneficios/consulta?ordenarPor=mesAno&direcao=desc
# URL(Atualizaçoes): http://www.portaltransparencia.gov.br/origem-dos-dados
# URL (Tabelas deste assunto): http://www.portaltransparencia.gov.br/download-de-dados

In [31]:
import requests
from bs4 import BeautifulSoup
import re

############# Codigo para buscar data da informacao (Producao) #################### 

URL = 'http://www.portaltransparencia.gov.br/beneficios/consulta?ordenarPor=mesAno&direcao=desc'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
tableDiv = soup.find_all('div', id="datas")
datinfo = re.sub("[^\d\.]", "", str(tableDiv))[26:]
print('Data da Info: ', datinfo)

#Data baixar arquivo
datinfo_mes = datinfo[2:-4]
datinfo_ano = datinfo[4:]
dat_get_file = datinfo_ano+datinfo_mes
print('Data para baixar arquivo:',dat_get_file)

Data da Info:  01012019
Data para baixar arquivo: 201901


In [30]:
ANOMES_INFO = 201801

#http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/201801


var_url = 'http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/'+str(ANOMES_INFO)
print('URL para download dos dados:')
print(var_url)

print('Baixando dados...')
import urllib.request
urllib.request.urlretrieve(var_url, 'dados_bolsafam_' + str(ANOMES_INFO) + '.zip')

print('Descompactando dados...')
import zipfile
with zipfile.ZipFile('dados_bolsafam_' + str(ANOMES_INFO) + '.zip',"r") as zip_ref:
    zip_ref.extractall('dados_bolsafam_' + str(ANOMES_INFO))
print('Arquivos disponiveis')    

URL para download dos dados:
http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/201801
Baixando dados...
Descompactando dados...


In [3]:
# Verificar este nome pelo arquivo descompactado 
ANOMES_INFO = 201801
nm_file1 = str(ANOMES_INFO)+'_BolsaFamilia_Pagamentos.csv'


#-----------------------  Bolsa Familia ------------------------------
import pandas as pd
import dask.dataframe as dd
import chardet
with open('dados_bolsafam_' + str(ANOMES_INFO) +'/' + nm_file1, 'rb') as f:
    result = chardet.detect(f.readline())  # or readline if the file is large
    
    
df_00 = pd.read_csv('dados_bolsafam_' + str(ANOMES_INFO) +'/' + nm_file1, encoding=result['encoding'], error_bad_lines=False,sep=';', engine='python')
df_00['DT_INFO'] = ANOMES_INFO





FileNotFoundError: [Errno 2] No such file or directory: 'dados_bolsafam_201801/201801_BolsaFamilia_Pagamentos.csv'

In [42]:
df_00.describe()

,Ano/Mês Referência,Ano/Mês Competência,Código Município SIAFI,NIS Beneficiário,DT_INFO
npartitions=1,,,,,
,int64,int64,int64,int64,int64
,...,...,...,...,...


In [17]:
import chardet
import pandas as pd
ANOMES_INFO=201701
nm_file1 = str(ANOMES_INFO)+'_BolsaFamilia_Pagamentos.csv'
print('Decodificando dados...')
with open('dados_bolsafam_' + str(ANOMES_INFO) +'/' + nm_file1, 'rb') as f:
    result = chardet.detect(f.readline())  # or readline if the file is large
print('Carregando dados em memoria...')
df_00 = pd.read_csv('dados_bolsafam_' + str(ANOMES_INFO) +'/' + nm_file1, encoding=result['encoding'], error_bad_lines=False,sep=';', engine='python')


Decodificando dados...
Carregando dados em memoria...


In [21]:
extr = 'H'
encode_df = result['encoding']
df_00['DT_INFO'] = ANOMES_INFO
df_00['ID_PROC'] = extr

In [22]:
df_00.head()

,Ano/Mês Referência,Ano/Mês Competência,UF,Código Município SIAFI,Nome Município SIAFI,NIS Beneficiário,Nome Beneficiário,Valor Benefício,DT_INFO,ID_PROC
0,201701,201601,MA,723,BACABAL,16550200718,TEREZINHA DE FATIMA ESPINDOLA,"46,00",201701,H
1,201701,201601,SP,6639,LIMEIRA,16081867537,ANGELA MARIA MESSIAS MORAES,"46,00",201701,H
2,201701,201602,AC,109,MANCIO LIMA,16379762982,MARIA DEILDES BARBOSA MESQUITA,"163,00",201701,H
3,201701,201602,AC,151,PLACIDO DE CASTRO,16601338657,VAL JOSE ALMIRO DE MATTOS,"202,00",201701,H
4,201701,201602,AC,139,RIO BRANCO,20453008245,MARIA DAMIANA SILVA ARAUJO,"46,00",201701,H


In [32]:
def CetusLakeBeneficiosCidadaoBolsaFamilia(dataproc,ANOMES_INFO,exec_lake = 'prod'):
    
    #---- Pagina de Extracao -----------
    # http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos
    
    #---- Metadados do Arquivo ---------
    # http://www.portaltransparencia.gov.br/pagina-interna/603397-dicionario-de-dados-bolsa-familia-pagamentos
    
    
    # ----- Pegando a Data da Informacao e Gravando na Tabela -------------------
    import requests
    from bs4 import BeautifulSoup
    import re
    import dask.dataframe as dd
    
    if exec_lake == 'prod':
        print('CETUS --> Executando em Producao')
        extr = 'P'

        # Utilizado para producao (execucao semanal ou mensal)
        URL = 'http://www.portaltransparencia.gov.br/beneficios/consulta?ordenarPor=mesAno&direcao=desc'
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        tableDiv = soup.find_all('div', id="datas")
        datinfo = re.sub("[^\d\.]", "", str(tableDiv))[26:]
        print('Data da Info: ', datinfo)
        
        #Data baixar arquivo
        datinfo_mes = datinfo[2:-4]
        datinfo_ano = datinfo[4:]
        ANOMES_INFO = datinfo_ano+datinfo_mes
        print('Data para baixar arquivo:',ANOMES_INFO)
        # ----------------------------------------------------------------------------
    else:
        print('CETUS --> Executando Carga Historica')
        # Formato que pegamos no site quando no modo producao (DDMMAAAA)
        datinfo = str(ANOMES_INFO)
        extr = 'H'
 
   
    var_url = 'http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/'+str(ANOMES_INFO)
    print('URL para download dos dados:')
    print(var_url)

    print('Baixando dados...')
    import urllib.request
    urllib.request.urlretrieve(var_url, 'dados_bolsafam_' + str(ANOMES_INFO) + '.zip')

    print('Descompactando dados...')
    import zipfile
    with zipfile.ZipFile('dados_bolsafam_' + str(ANOMES_INFO) + '.zip',"r") as zip_ref:
        zip_ref.extractall('dados_bolsafam_' + str(ANOMES_INFO))
    print('Arquivos disponiveis') 

    #-----------------------  Bolsa Familia ------------------------------
    # Verificar este nome pelo arquivo descompactado 
    nm_file1 = str(ANOMES_INFO)+'_BolsaFamilia_Pagamentos.csv'

    #-----------------------  Bolsa Familia ------------------------------
    import pandas as pd
    import chardet
    print('Decodificando dados...')
    with open('dados_bolsafam_' + str(ANOMES_INFO) +'/' + nm_file1, 'rb') as f:
        result = chardet.detect(f.readline())  # or readline if the file is large
    print('Carregando dados em memoria...')
    df_00 = pd.read_csv('dados_bolsafam_' + str(ANOMES_INFO) +'/' + nm_file1, encoding=result['encoding'], error_bad_lines=False,sep=';', engine='python')
    encode_df = result['encoding']
    df_00['DT_INFO'] = ANOMES_INFO
    df_00['ID_PROC'] = extr
    return df_00,encode_df,datinfo

#names=["a", "b"], dtype={"a": np.str, "b": np.float64}

def CetusSalvaCSVS3(dataframe,nm_s3_file,s3_path,encode_df):

    import s3fs
    S3fs = s3fs.S3FileSystem()

    # Nome do arquivo a ser salvo no S3
    #nm_s3_file = 'OrcamentoDespesas_'+str(dtref)+'.csv'

    # Caminho no lake (S3)
    #s3_path = 'projeto-cetus/datalake/OrcamentoDespesas/dados/'

    bytes_to_write = dataframe.to_csv(sep = ',',header=True, index=False, index_label=None,encoding=encode_df).encode()
    with S3fs.open(s3_path+nm_s3_file, 'wb') as f:
        f.write(bytes_to_write)

        
     
        
        
        
        
def CetusRefVecDays(inicio,fim):
     import datetime
     start = datetime.datetime.strptime(inicio, "%Y%m%d")
     end = datetime.datetime.strptime(fim, "%Y%m%d")
     date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
     #print('Data inicial: ',start.strftime("%Y%m%d") ) 
     #print('Data final: ',end.strftime("%Y%m%d") ) 
     #print('Quantidade de Partições: ',len(date_generated)) 
     return date_generated



### Carga Histórica

In [33]:
vetor_safras_aux = CetusRefVecDays('20170101','20181231') 
#s3_path = 'projeto-bigdata-cetus/datalake/xxxxxxxxxxxx/'

anomes = []
for data_ref in vetor_safras_aux:    
    anomes.append(data_ref.strftime("%Y%m"))
    
vetor_safras = list(dict.fromkeys(anomes))


import datetime
import time
now = datetime.datetime.now() - datetime.timedelta(0,0,0,0,3)
anomesdia = now.strftime("%Y%m%d")
diamesano = now.strftime("%d%m%Y")
anoref = now.strftime("%Y")


for anomes in vetor_safras:
    start_time = time.clock()
    print('Cetus - Carga Histórica Ref ---->', anomes)
    dados,encode_df,datinfo = CetusLakeBeneficiosCidadaoBolsaFamilia(diamesano,anomes,exec_lake = 'Hist')
    nm_s3_file = 'BC_Lake_BolFamPag_'+str(datinfo)+'.csv'
    s3_path = 'projeto-bigdata-cetus/datalake/BeneficiosCidadaos/'
    print('Salvando tabela no lake ---> ',nm_s3_file)
    CetusSalvaCSVS3(dados,nm_s3_file,s3_path,encode_df)
    fin_time = time.clock()
    delta = round(((fin_time-start_time)/60),4)
    print('Tempo(min) para extracao ----> ',delta)

Cetus - Carga Histórica Ref ----> 201701
CETUS --> Executando Carga Historica
URL para download dos dados:
http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/201701
Baixando dados...
Descompactando dados...
Arquivos disponiveis
Decodificando dados...
Carregando dados em memoria...
Salvando tabela no lake --->  BC_Lake_BolFamPag_201701.csv
Tempo(min) para extracao ---->  3.5207
Cetus - Carga Histórica Ref ----> 201702
CETUS --> Executando Carga Historica
URL para download dos dados:
http://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/201702
Baixando dados...


KeyboardInterrupt: 

In [25]:
encode_df

'ISO-8859-1'

In [28]:
CetusSalvaCSVS3(dados,nm_s3_file,s3_path,'utf-8')

In [ ]:
from io import StringIO
import boto3

csv_buffer = StringIO()
dados.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('projeto-bigdata-cetus', 'teste_20190313.csv').put(Body=csv_buffer.getvalue())